In [1]:
import pandas as pd
import numpy as np
from utils_clean import *
import re
import matplotlib.pyplot as plt
import seaborn as sns

Filtering for top 100 players only and non-challenger matches

In [2]:
dtypes={'tourney_id': str,
        'draw_size': str,
        'winner_id': 'Int64',
        'loser_id': 'Int64',
        'winner_seed':  str,
        'winner_entry': str,
        'loser_entry': str,
        'loser_seed':  str,
       }

dtypes={**dtypes,**{'W'+str(i+1):'Int64' for i in range(5)},**{'L'+str(i+1):'Int64' for i in range(5)}}
df=pd.read_csv('csvs/atp_matches_time.csv',dtype=dtypes, date_format='%Y%m%d',parse_dates=['tourney_date'])

In [3]:
df

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,W4,W5,L1,L2,L3,L4,L5,n_set,l_min,w_min
0,1992-301,Auckland,Hard,32,A,1992-01-06,1,101797,NaN,NaN,...,<NA>,<NA>,6,6,<NA>,<NA>,<NA>,2.0,NaN,0.0
1,1992-301,Auckland,Hard,32,A,1992-01-06,2,101205,NaN,NaN,...,<NA>,<NA>,3,4,<NA>,<NA>,<NA>,2.0,NaN,0.0
2,1992-301,Auckland,Hard,32,A,1992-01-06,3,101368,NaN,NaN,...,<NA>,<NA>,4,2,<NA>,<NA>,<NA>,2.0,NaN,0.0
3,1992-301,Auckland,Hard,32,A,1992-01-06,4,100772,NaN,WC,...,<NA>,<NA>,6,5,<NA>,<NA>,<NA>,2.0,NaN,0.0
4,1992-301,Auckland,Hard,32,A,1992-01-06,5,101532,4.0,NaN,...,<NA>,<NA>,4,6,3,<NA>,<NA>,3.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295042,2024-8266,Augsburg CH,Clay,32,Challenger,2024-05-20,296,109054,NaN,NaN,...,<NA>,<NA>,3,2,<NA>,<NA>,<NA>,2.0,252.0,250.0
295043,2024-8266,Augsburg CH,Clay,32,Challenger,2024-05-20,297,207608,1,NaN,...,<NA>,<NA>,6,4,<NA>,<NA>,<NA>,2.0,226.0,223.0
295044,2024-8266,Augsburg CH,Clay,32,Challenger,2024-05-20,298,209284,8,NaN,...,<NA>,<NA>,6,2,5,<NA>,<NA>,3.0,277.0,350.0
295045,2024-8266,Augsburg CH,Clay,32,Challenger,2024-05-20,299,207608,1,NaN,...,<NA>,<NA>,5,5,<NA>,<NA>,<NA>,2.0,325.0,325.0


In [62]:
df.columns

Index(['tourney_id', 'tourney_name', 'surface', 'draw_size', 'tourney_level',
       'tourney_date', 'match_num', 'winner_id', 'winner_seed', 'winner_entry',
       'winner_name', 'winner_hand', 'winner_ht', 'winner_ioc', 'winner_age',
       'loser_id', 'loser_seed', 'loser_entry', 'loser_name', 'loser_hand',
       'loser_ht', 'loser_ioc', 'loser_age', 'score', 'best_of', 'round',
       'minutes', 'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon',
       'w_SvGms', 'w_bpSaved', 'w_bpFaced', 'l_ace', 'l_df', 'l_svpt',
       'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved', 'l_bpFaced',
       'winner_rank', 'winner_rank_points', 'loser_rank', 'loser_rank_points',
       'retirement', 'W1', 'W2', 'W3', 'W4', 'W5', 'L1', 'L2', 'L3', 'L4',
       'L5', 'n_set', 'l_min', 'w_min'],
      dtype='object')

In [63]:
df=df.loc[(~df.winner_rank.isna())&(~df.loser_rank.isna()),]
df=df.loc[(df.winner_rank<101)|(df.loser_rank <101),]
df=df.loc[df.tourney_level.isin(['A','Grand Slam','Masters','F']),]

In [64]:
df

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,W4,W5,L1,L2,L3,L4,L5,n_set,l_min,w_min
0,1992-301,Auckland,Hard,32,A,1992-01-06,1,101797,NaN,NaN,...,<NA>,<NA>,6,6,<NA>,<NA>,<NA>,2.0,NaN,0.0
1,1992-301,Auckland,Hard,32,A,1992-01-06,2,101205,NaN,NaN,...,<NA>,<NA>,3,4,<NA>,<NA>,<NA>,2.0,NaN,0.0
2,1992-301,Auckland,Hard,32,A,1992-01-06,3,101368,NaN,NaN,...,<NA>,<NA>,4,2,<NA>,<NA>,<NA>,2.0,NaN,0.0
3,1992-301,Auckland,Hard,32,A,1992-01-06,4,100772,NaN,WC,...,<NA>,<NA>,6,5,<NA>,<NA>,<NA>,2.0,NaN,0.0
4,1992-301,Auckland,Hard,32,A,1992-01-06,5,101532,4.0,NaN,...,<NA>,<NA>,4,6,3,<NA>,<NA>,3.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294993,2024-7694,Lyon,Clay,32,A,2024-05-20,296,209260,NaN,NaN,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.0,311.0,237.0
294994,2024-7694,Lyon,Clay,32,A,2024-05-20,297,144869,6,NaN,...,<NA>,<NA>,4,1,<NA>,<NA>,<NA>,2.0,278.0,259.0
294995,2024-7694,Lyon,Clay,32,A,2024-05-20,298,208659,NaN,WC,...,<NA>,<NA>,4,5,<NA>,<NA>,<NA>,2.0,260.0,197.0
294996,2024-7694,Lyon,Clay,32,A,2024-05-20,299,144869,6,NaN,...,<NA>,<NA>,2,4,<NA>,<NA>,<NA>,2.0,237.0,359.0


In [65]:
indices=['tourney_name', 'surface', 'draw_size','tourney_level',
       'tourney_date', 'winner_id', 'winner_seed',
       'winner_name', 'winner_ioc', 'winner_age',
       'loser_id', 'loser_seed', 'loser_name', 'loser_ioc', 'loser_age', 'score', 'best_of', 'round',
       'minutes', 'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon',
       'w_SvGms', 'w_bpSaved', 'w_bpFaced', 'l_ace', 'l_df', 'l_svpt',
       'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved', 'l_bpFaced',
       'winner_rank', 'loser_rank',
       'retirement', 'W1', 'W2', 'W3', 'W4', 'W5', 'L1', 'L2', 'L3', 'L4',
       'L5', 'n_set', 'l_min', 'w_min']

In [66]:
df=df.loc[:,indices]

In [67]:
df

,tourney_name,surface,draw_size,tourney_level,tourney_date,winner_id,winner_seed,winner_name,winner_ioc,winner_age,...,W4,W5,L1,L2,L3,L4,L5,n_set,l_min,w_min
0,Auckland,Hard,32,A,1992-01-06,101797,NaN,Jacco Eltingh,NED,21.3,...,<NA>,<NA>,6,6,<NA>,<NA>,<NA>,2.0,NaN,0.0
1,Auckland,Hard,32,A,1992-01-06,101205,NaN,Grant Connell,CAN,26.1,...,<NA>,<NA>,3,4,<NA>,<NA>,<NA>,2.0,NaN,0.0
2,Auckland,Hard,32,A,1992-01-06,101368,NaN,Christian Miniussi,ARG,24.5,...,<NA>,<NA>,4,2,<NA>,<NA>,<NA>,2.0,NaN,0.0
3,Auckland,Hard,32,A,1992-01-06,100772,NaN,Kelly Evernden,NZL,30.2,...,<NA>,<NA>,6,5,<NA>,<NA>,<NA>,2.0,NaN,0.0
4,Auckland,Hard,32,A,1992-01-06,101532,4.0,Francisco Clavet,ESP,23.2,...,<NA>,<NA>,4,6,3,<NA>,<NA>,3.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294993,Lyon,Clay,32,A,2024-05-20,209260,NaN,Luciano Darderi,ITA,22.2,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.0,311.0,237.0
294994,Lyon,Clay,32,A,2024-05-20,144869,6,Tomas Martin Etcheverry,ARG,24.8,...,<NA>,<NA>,4,1,<NA>,<NA>,<NA>,2.0,278.0,259.0
294995,Lyon,Clay,32,A,2024-05-20,208659,NaN,Giovanni Mpetshi Perricard,FRA,20.8,...,<NA>,<NA>,4,5,<NA>,<NA>,<NA>,2.0,260.0,197.0
294996,Lyon,Clay,32,A,2024-05-20,144869,6,Tomas Martin Etcheverry,ARG,24.8,...,<NA>,<NA>,2,4,<NA>,<NA>,<NA>,2.0,237.0,359.0


In [68]:
col_names=[]
for i in df.columns:
    w1=re.sub(r'winner',r'p1',i)
    w2=re.sub(r'^w_',r'p1_',w1)
    w3=re.sub(r'W(\d)',r'p1_sets_\1',w2)
    l1=re.sub(r'loser',r'p2',w3)
    l2=re.sub(r'^l_',r'p2_',l1)
    l3=re.sub(r'L(\d)',r'p2_sets_\1',l2)
    col_names.append(l3)
df.columns=col_names

In [69]:
df

,tourney_name,surface,draw_size,tourney_level,tourney_date,p1_id,p1_seed,p1_name,p1_ioc,p1_age,...,p1_sets_4,p1_sets_5,p2_sets_1,p2_sets_2,p2_sets_3,p2_sets_4,p2_sets_5,n_set,p2_min,p1_min
0,Auckland,Hard,32,A,1992-01-06,101797,NaN,Jacco Eltingh,NED,21.3,...,<NA>,<NA>,6,6,<NA>,<NA>,<NA>,2.0,NaN,0.0
1,Auckland,Hard,32,A,1992-01-06,101205,NaN,Grant Connell,CAN,26.1,...,<NA>,<NA>,3,4,<NA>,<NA>,<NA>,2.0,NaN,0.0
2,Auckland,Hard,32,A,1992-01-06,101368,NaN,Christian Miniussi,ARG,24.5,...,<NA>,<NA>,4,2,<NA>,<NA>,<NA>,2.0,NaN,0.0
3,Auckland,Hard,32,A,1992-01-06,100772,NaN,Kelly Evernden,NZL,30.2,...,<NA>,<NA>,6,5,<NA>,<NA>,<NA>,2.0,NaN,0.0
4,Auckland,Hard,32,A,1992-01-06,101532,4.0,Francisco Clavet,ESP,23.2,...,<NA>,<NA>,4,6,3,<NA>,<NA>,3.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294993,Lyon,Clay,32,A,2024-05-20,209260,NaN,Luciano Darderi,ITA,22.2,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.0,311.0,237.0
294994,Lyon,Clay,32,A,2024-05-20,144869,6,Tomas Martin Etcheverry,ARG,24.8,...,<NA>,<NA>,4,1,<NA>,<NA>,<NA>,2.0,278.0,259.0
294995,Lyon,Clay,32,A,2024-05-20,208659,NaN,Giovanni Mpetshi Perricard,FRA,20.8,...,<NA>,<NA>,4,5,<NA>,<NA>,<NA>,2.0,260.0,197.0
294996,Lyon,Clay,32,A,2024-05-20,144869,6,Tomas Martin Etcheverry,ARG,24.8,...,<NA>,<NA>,2,4,<NA>,<NA>,<NA>,2.0,237.0,359.0


In [70]:
def n_m(df):
    #Evaluates how many matches played by each player
    sr1=df.groupby('p2_name').size()
    sr2=df.groupby('p1_name').size()
    return sr1.add(sr2, fill_value=0)

In [71]:
while not (n_m(df)>14).all():
    X=n_m(df)
    t=(X.loc[X>14]).index
    df=df.loc[df.p1_name.isin(t) & df.p2_name.isin(t)]

In [97]:
df

,tourney_name,surface,draw_size,tourney_level,tourney_date,score,best_of,round,minutes,retirement,...,p2_bpSaved,p2_bpFaced,p2_rank,p2_sets_1,p2_sets_2,p2_sets_3,p2_sets_4,p2_sets_5,p2_min,outcome
0,Auckland,Hard,32,A,1992-01-06,7-6(5) 7-6(3),3,R32,112.0,False,...,4.0,4.0,8.0,6,6,<NA>,<NA>,<NA>,NaN,True
1,Auckland,Hard,32,A,1992-01-06,6-3 6-4,3,R32,68.0,False,...,5.0,8.0,101.0,3,4,<NA>,<NA>,<NA>,NaN,True
5,Auckland,Hard,32,A,1992-01-06,6-2 1-6 6-2,3,R32,93.0,False,...,7.0,13.0,73.0,2,6,2,<NA>,<NA>,NaN,True
6,Auckland,Hard,32,A,1992-01-06,6-1 5-7 7-6(4),3,R32,128.0,False,...,7.0,11.0,76.0,1,7,6,<NA>,<NA>,NaN,True
7,Auckland,Hard,32,A,1992-01-06,6-1 6-1,3,R32,80.0,False,...,NaN,NaN,42.0,1,1,<NA>,<NA>,<NA>,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294990,Lyon,Clay,32,A,2024-05-20,2-6 6-4 6-4,3,R16,124.0,False,...,1.0,5.0,17.0,6,4,4,<NA>,<NA>,NaN,True
294991,Lyon,Clay,32,A,2024-05-20,6-7(4) 7-6(4) 6-3,3,QF,191.0,False,...,3.0,8.0,57.0,7,6,3,<NA>,<NA>,163.0,True
294993,Lyon,Clay,32,A,2024-05-20,NaN,3,QF,0.0,True,...,NaN,NaN,73.0,<NA>,<NA>,<NA>,<NA>,<NA>,311.0,True
294994,Lyon,Clay,32,A,2024-05-20,6-4 6-1,3,QF,100.0,False,...,1.0,5.0,67.0,4,1,<NA>,<NA>,<NA>,278.0,True


In [73]:
col_names=df.columns
n_t=[]
w_t=[]
l_t=[]
for i in col_names:
    if bool(re.search(r'p1', i)):
        w_t.append(i)
    elif bool(re.search(r'p2', i)):
        l_t.append(i)
    else:
        n_t.append(i)

In [74]:
df=df.loc[:,n_t+w_t+l_t]
col_names=df.columns
df['outcome']=True


In [91]:
def converter(x):
    if bool(np.random.randint(2)):
        return x
    else:
        a=x.loc[n_t+l_t+w_t]
        a['outcome']=False
        a.index=n_t+w_t+l_t+['outcome']
        return a

In [92]:
df1=df.apply(converter,axis=1)

In [93]:
df1

,tourney_name,surface,draw_size,tourney_level,tourney_date,score,best_of,round,minutes,retirement,...,p2_bpSaved,p2_bpFaced,p2_rank,p2_sets_1,p2_sets_2,p2_sets_3,p2_sets_4,p2_sets_5,p2_min,outcome
0,Auckland,Hard,32,A,1992-01-06,7-6(5) 7-6(3),3,R32,112.0,False,...,4.0,4.0,8.0,6,6,<NA>,<NA>,<NA>,NaN,True
1,Auckland,Hard,32,A,1992-01-06,6-3 6-4,3,R32,68.0,False,...,5.0,8.0,101.0,3,4,<NA>,<NA>,<NA>,NaN,True
5,Auckland,Hard,32,A,1992-01-06,6-2 1-6 6-2,3,R32,93.0,False,...,0.0,4.0,53.0,6,1,6,<NA>,<NA>,0.0,False
6,Auckland,Hard,32,A,1992-01-06,6-1 5-7 7-6(4),3,R32,128.0,False,...,7.0,11.0,76.0,1,7,6,<NA>,<NA>,NaN,True
7,Auckland,Hard,32,A,1992-01-06,6-1 6-1,3,R32,80.0,False,...,NaN,NaN,42.0,1,1,<NA>,<NA>,<NA>,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294990,Lyon,Clay,32,A,2024-05-20,2-6 6-4 6-4,3,R16,124.0,False,...,1.0,5.0,17.0,6,4,4,<NA>,<NA>,NaN,True
294991,Lyon,Clay,32,A,2024-05-20,6-7(4) 7-6(4) 6-3,3,QF,191.0,False,...,13.0,17.0,19.0,6,7,6,<NA>,<NA>,69.0,False
294993,Lyon,Clay,32,A,2024-05-20,NaN,3,QF,0.0,True,...,NaN,NaN,73.0,<NA>,<NA>,<NA>,<NA>,<NA>,311.0,True
294994,Lyon,Clay,32,A,2024-05-20,6-4 6-1,3,QF,100.0,False,...,1.0,5.0,67.0,4,1,<NA>,<NA>,<NA>,278.0,True


In [104]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [103]:
clf = RandomForestClassifier(n_estimators=10, max_depth=None, min_samples_split=2, random_state=0)

In [109]:
c_ind=['surface', 'draw_size',
       'p1_name',  'p1_age','p1_min',
        'p2_name', 'p2_age', 'p2_min']

In [142]:
X=df1.loc[:,['draw_size','p1_age','p1_min','p2_age', 'p2_min']]
X=X.fillna(0)

In [143]:
BT=pd.get_dummies(df1['p1_name'],dtype=float)-pd.get_dummies(df1['p2_name'],dtype=float)
BT=BT.fillna(0)
S=pd.get_dummies(df1['surface'],dtype=float)

In [144]:
X=X.join(BT).join(S)

In [145]:
X

,draw_size,p1_age,p1_min,p2_age,p2_min,Aaron Krickstein,Adrian Garcia,Adrian Mannarino,Adrian Menendez Maceiras,Adrian Ungur,...,Yuri Schukin,Zack Fleishman,Ze Zhang,Zeljko Krajan,Zhizhen Zhang,Zizou Bergs,Carpet,Clay,Grass,Hard
0,32,21.3,0.0,26.7,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,32,26.1,0.0,21.5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5,32,26.3,0.0,24.2,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6,32,25.0,0.0,20.6,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7,32,21.9,0.0,20.3,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294990,32,30.0,154.0,25.9,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
294991,32,25.5,163.0,26.9,69.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
294993,32,22.2,237.0,28.8,311.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
294994,32,24.8,259.0,30.0,278.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [152]:
X.shape

(87065, 885)

In [162]:
p_train=0.8
ind_t=round(p_train*(X.shape[0]))
X_train=X.iloc[:ind_p,:]
X_val=X.iloc[ind_p:,:]
y_train=df1.outcome.iloc[:ind_p]
y_val=df1.outcome.iloc[ind_p:]

In [165]:
X_train

,draw_size,p1_age,p1_min,p2_age,p2_min,Aaron Krickstein,Adrian Garcia,Adrian Mannarino,Adrian Menendez Maceiras,Adrian Ungur,...,Yuri Schukin,Zack Fleishman,Ze Zhang,Zeljko Krajan,Zhizhen Zhang,Zizou Bergs,Carpet,Clay,Grass,Hard
0,32,21.3,0.0,26.7,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,32,26.1,0.0,21.5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5,32,26.3,0.0,24.2,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6,32,25.0,0.0,20.6,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7,32,21.9,0.0,20.3,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215868,128,26.7,80.0,24.8,80.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
215869,128,28.5,130.0,32.1,117.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
215871,128,30.0,165.0,33.2,131.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
215873,128,36.8,260.0,30.9,130.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
